In [1]:
### import relevant packages

from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments, AutoModelForCausalLM
from datasets import Dataset

from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM


In [10]:
## mention a cache dir if you want to save a local file
cache_dir='/scratch/gpfs/ap34/hf_models'

model_name = "/scratch/gpfs/ARORA/" ### change here for other models
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir=cache_dir)

tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)  ### load tokenizer
tokenizer.pad_token = tokenizer.eos_token

In [11]:
data = load_dataset('openai/gsm8k', 'main', cache_dir=cache_dir)

Using the latest cached version of the dataset since openai/gsm8k couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'main' at /scratch/gpfs/ap34/hf_models/openai___gsm8k/main/0.0.0/e53f048856ff4f594e959d75785d2c2d37b678ee (last modified on Thu Apr 10 08:47:38 2025).


In [17]:
train_data = data['train']
test_data = data['test']

def create_conversation(d):
    text = '### Question:\n' + str(d['question']) + '\n\n### Solution:\n' + str(d['answer'])
    return {'text': text}

conversation_train_data = []
for d in train_data:
    conversation_train_data += [create_conversation(d)]
conversation_train_data = Dataset.from_list(conversation_train_data)



'### Question:\nNatalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?\n\n### Solution:\nNatalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'

In [22]:
# define the training arguments
output_dir='result'

num_train_epochs=3
per_device_train_batch_size=1
gradient_accumulation_steps=64
gradient_checkpointing=False
optim = 'adamw_torch'

logging_steps=10
save_strategy='epoch'
bf16='True'

learning_rate=1e-4
weight_decay=0.0

max_sequence_length=512

warmup_ratio=0.03
lr_scheduler_type='cosine'


In [ ]:
response_template = "### Solution:\n" 

collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

args = SFTConfig(
    output_dir=output_dir,  # directory to save and repository id
    num_train_epochs=num_train_epochs,  # number of training epochs
    per_device_train_batch_size=per_device_train_batch_size,  # batch size per device during training
    gradient_accumulation_steps=gradient_accumulation_steps,  # number of steps before performing a backward/update pass
    gradient_checkpointing=gradient_checkpointing,  # use gradient checkpointing to save memory
    optim=optim,  # use fused adamw optimizer
    logging_steps=logging_steps,  # log every 10 steps
    save_strategy=save_strategy,  # save checkpoint every epoch
    bf16=bf16,  # use bfloat16 precision
    learning_rate=learning_rate,  # learning rate, based on QLoRA paper
    weight_decay=weight_decay,
    max_seq_length=max_sequence_length,
    warmup_ratio=warmup_ratio,  # warmup ratio based on QLoRA paper
    lr_scheduler_type=lr_scheduler_type,  # use constant learning rate scheduler
    report_to='tensorboard',
)

config = {}
config['model'] = model
config['processing_class'] = tokenizer
config['train_dataset'] = conversation_train_data

config['args'] = args
config['data_collator'] = collator


trainer = SFTTrainer(**config)

train_result = trainer.train()
metrics = train_result.metrics

metrics["train_samples"] = len(conversation_train_data)

trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()


trainer.save_model(output_dir)  # Saves model & weights

# Explicitly save the tokenizer
tokenizer.save_pretrained(output_dir)


In [28]:
## Now evaluate by generating the responses
from transformers import GenerationConfig

config = GenerationConfig(max_length=max_sequence_length)


def create_question(d):
    text = '### Question:\n' + str(d['question']) + '\n\n### Solution:\n' 
    return {'text': text}


conversation_test_data = []
for d in test_data:
    conversation_test_data += [create_question(d)]
conversation_test_data =  Dataset.from_list(conversation_test_data)


correct = 0.
total = 0.

for d in conversation_test_data:
    





WARNING 04-22 17:54:35 _core_ext.py:180] Failed to import from vllm._core_C with ImportError('/scratch/gpfs/ap34/miniconda_envs/envs/MOSS/lib/python3.10/site-packages/vllm/_core_C.abi3.so: undefined symbol: _ZNK3c105Error4whatEv')


RuntimeError: Tried to instantiate class '_core_C.ScalarType', but it does not exist! Ensure that it is registered via torch::class_